In [1]:
import os
import sys
mod_path = os.path.abspath(os.path.join('..'))
if mod_path not in sys.path:
    sys.path.append(mod_path)

In [17]:
import pickle
from pathlib import Path
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from pprint import pprint as pp
import pandas as pd

In [4]:
from tools.feature_format import featureFormat, targetFeatureSplit
from final_project.tester import dump_classifier_and_data

In [5]:
pwd_ = %pwd
pwd_ = Path(pwd_)
pwd_.joinpath('tools/feature_format.py')

WindowsPath('E:/Users/Trenton J. McKinney/PycharmProjects/ud120-projects/tools/feature_format.py')

# Task 1: Select Features
#### features_list is a list of strings, each of which is a feature name.
#### The first feature must be "poi".

In [6]:
financial_features = ['salary', 'deferral_payments', 'total_payments', 'loan_advances', 'bonus', 'restricted_stock_deferred',
                      'deferred_income', 'total_stock_value', 'expenses', 'exercised_stock_options', 'other',
                      'long_term_incentive', 'restricted_stock', 'director_fees']

email_features = ['to_messages', 'email_address', 'from_poi_to_this_person', 'from_messages', 'from_this_person_to_poi', 'shared_receipt_with_poi']

poi_label = ['poi']

In [12]:
feature_list = []
feature_list.extend(poi_label)
feature_list.extend(financial_features)
feature_list.extend(email_features)
feature_list

['poi',
 'salary',
 'deferral_payments',
 'total_payments',
 'loan_advances',
 'bonus',
 'restricted_stock_deferred',
 'deferred_income',
 'total_stock_value',
 'expenses',
 'exercised_stock_options',
 'other',
 'long_term_incentive',
 'restricted_stock',
 'director_fees',
 'to_messages',
 'email_address',
 'from_poi_to_this_person',
 'from_messages',
 'from_this_person_to_poi',
 'shared_receipt_with_poi']

## Load Dataset

In [59]:
# Load the dictionary containing the dataset
with open('final_project/final_project_dataset_unix.pkl', 'rb') as data_file:
    data_dict = pickle.load(data_file)

## Explore Dataset

In [60]:
df = pd.DataFrame.from_dict(data_dict, orient='index')
pd.set_option('max_rows', 150)
pd.set_option('max_columns', 21)
df

,salary,to_messages,deferral_payments,total_payments,loan_advances,bonus,email_address,restricted_stock_deferred,deferred_income,total_stock_value,expenses,from_poi_to_this_person,exercised_stock_options,from_messages,other,from_this_person_to_poi,poi,long_term_incentive,shared_receipt_with_poi,restricted_stock,director_fees
ALLEN PHILLIP K,201955,2902,2869717,4484442,NaN,4175000,phillip.allen@enron.com,-126027,-3081055,1729541,13868,47,1729541,2195,152,65,False,304805,1407,126027,NaN
BADUM JAMES P,NaN,NaN,178980,182466,NaN,NaN,NaN,NaN,NaN,257817,3486,NaN,257817,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN
BANNANTINE JAMES M,477,566,NaN,916197,NaN,NaN,james.bannantine@enron.com,-560222,-5104,5243487,56301,39,4046157,29,864523,0,False,NaN,465,1757552,NaN
BAXTER JOHN C,267102,NaN,1295738,5634343,NaN,1200000,NaN,NaN,-1386055,10623258,11200,NaN,6680544,NaN,2660303,NaN,False,1586055,NaN,3942714,NaN
BAY FRANKLIN R,239671,NaN,260455,827696,NaN,400000,frank.bay@enron.com,-82782,-201641,63014,129142,NaN,NaN,NaN,69,NaN,False,NaN,NaN,145796,NaN
BAZELIDES PHILIP J,80818,NaN,684694,860136,NaN,NaN,NaN,NaN,NaN,1599641,NaN,NaN,1599641,NaN,874,NaN,False,93750,NaN,NaN,NaN
BECK SALLY W,231330,7315,NaN,969068,NaN,700000,sally.beck@enron.com,NaN,NaN,126027,37172,144,NaN,4343,566,386,False,NaN,2639,126027,NaN
BELDEN TIMOTHY N,213999,7991,2144013,5501630,NaN,5249999,tim.belden@enron.com,NaN,-2334434,1110705,17355,228,953136,484,210698,108,True,NaN,5521,157569,NaN
BELFER ROBERT,NaN,NaN,-102500,102500,NaN,NaN,NaN,44093,NaN,-44093,NaN,NaN,3285,NaN,NaN,NaN,False,NaN,NaN,NaN,3285
BERBERIAN DAVID,216582,NaN,NaN,228474,NaN,NaN,david.berberian@enron.com,NaN,NaN,2493616,11892,NaN,1624396,NaN,NaN,NaN,False,NaN,NaN,869220,NaN


In [61]:
df.describe()

,salary,to_messages,deferral_payments,total_payments,loan_advances,bonus,email_address,restricted_stock_deferred,deferred_income,total_stock_value,expenses,from_poi_to_this_person,exercised_stock_options,from_messages,other,from_this_person_to_poi,poi,long_term_incentive,shared_receipt_with_poi,restricted_stock,director_fees
count,146,146,146,146,146,146,146,146,146,146,146,146,146,146,146,146,146,146,146,146,146
unique,95,87,40,126,5,42,112,19,45,125,95,58,102,65,93,42,2,53,84,98,18
top,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN
freq,51,60,107,21,142,64,35,128,97,20,51,60,44,60,53,60,128,80,60,36,129


In [62]:
list(df)

['salary',
 'to_messages',
 'deferral_payments',
 'total_payments',
 'loan_advances',
 'bonus',
 'email_address',
 'restricted_stock_deferred',
 'deferred_income',
 'total_stock_value',
 'expenses',
 'from_poi_to_this_person',
 'exercised_stock_options',
 'from_messages',
 'other',
 'from_this_person_to_poi',
 'poi',
 'long_term_incentive',
 'shared_receipt_with_poi',
 'restricted_stock',
 'director_fees']

In [63]:
list(df.index.values)

['ALLEN PHILLIP K',
 'BADUM JAMES P',
 'BANNANTINE JAMES M',
 'BAXTER JOHN C',
 'BAY FRANKLIN R',
 'BAZELIDES PHILIP J',
 'BECK SALLY W',
 'BELDEN TIMOTHY N',
 'BELFER ROBERT',
 'BERBERIAN DAVID',
 'BERGSIEKER RICHARD P',
 'BHATNAGAR SANJAY',
 'BIBI PHILIPPE A',
 'BLACHMAN JEREMY M',
 'BLAKE JR. NORMAN P',
 'BOWEN JR RAYMOND M',
 'BROWN MICHAEL',
 'BUCHANAN HAROLD G',
 'BUTTS ROBERT H',
 'BUY RICHARD B',
 'CALGER CHRISTOPHER F',
 'CARTER REBECCA C',
 'CAUSEY RICHARD A',
 'CHAN RONNIE',
 'CHRISTODOULOU DIOMEDES',
 'CLINE KENNETH W',
 'COLWELL WESLEY',
 'CORDES WILLIAM R',
 'COX DAVID',
 'CUMBERLAND MICHAEL S',
 'DEFFNER JOSEPH M',
 'DELAINEY DAVID W',
 'DERRICK JR. JAMES V',
 'DETMERING TIMOTHY J',
 'DIETRICH JANET R',
 'DIMICHELE RICHARD G',
 'DODSON KEITH',
 'DONAHUE JR JEFFREY M',
 'DUNCAN JOHN H',
 'DURAN WILLIAM D',
 'ECHOLS JOHN B',
 'ELLIOTT STEVEN',
 'FALLON JAMES B',
 'FASTOW ANDREW S',
 'FITZGERALD JAY L',
 'FOWLER PEGGY',
 'FOY JOE',
 'FREVERT MARK A',
 'FUGH JOHN L',
 'GAHN 

In [64]:
df.loc['LOCKHART EUGENE E']

salary                         NaN
to_messages                    NaN
deferral_payments              NaN
total_payments                 NaN
loan_advances                  NaN
bonus                          NaN
email_address                  NaN
restricted_stock_deferred      NaN
deferred_income                NaN
total_stock_value              NaN
expenses                       NaN
from_poi_to_this_person        NaN
exercised_stock_options        NaN
from_messages                  NaN
other                          NaN
from_this_person_to_poi        NaN
poi                          False
long_term_incentive            NaN
shared_receipt_with_poi        NaN
restricted_stock               NaN
director_fees                  NaN
Name: LOCKHART EUGENE E, dtype: object

In [65]:
df.loc['THE TRAVEL AGENCY IN THE PARK']

salary                          NaN
to_messages                     NaN
deferral_payments               NaN
total_payments               362096
loan_advances                   NaN
bonus                           NaN
email_address                   NaN
restricted_stock_deferred       NaN
deferred_income                 NaN
total_stock_value               NaN
expenses                        NaN
from_poi_to_this_person         NaN
exercised_stock_options         NaN
from_messages                   NaN
other                        362096
from_this_person_to_poi         NaN
poi                           False
long_term_incentive             NaN
shared_receipt_with_poi         NaN
restricted_stock                NaN
director_fees                   NaN
Name: THE TRAVEL AGENCY IN THE PARK, dtype: object

In [66]:
df.loc['TOTAL']

salary                        26704229
to_messages                        NaN
deferral_payments             32083396
total_payments               309886585
loan_advances                 83925000
bonus                         97343619
email_address                      NaN
restricted_stock_deferred     -7576788
deferred_income              -27992891
total_stock_value            434509511
expenses                       5235198
from_poi_to_this_person            NaN
exercised_stock_options      311764000
from_messages                      NaN
other                         42667589
from_this_person_to_poi            NaN
poi                              False
long_term_incentive           48521928
shared_receipt_with_poi            NaN
restricted_stock             130322299
director_fees                  1398517
Name: TOTAL, dtype: object

# Task 2: Remove Outliers

In [67]:
outliers = ['THE TRAVEL AGENCY IN THE PARK', 'LOCKHART EUGENE E', 'TOTAL']
for value in outliers:
    del data_dict[value]

In [68]:
len(data_dict)

143